In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Video
from tqdm import tnrange, tqdm_notebook
import nd2reader
import util

hv.extension("matplotlib")

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=matplotlib.mplDeprecation)

In [ ]:
import io
import imageio
import PIL

# FROM: https://gist.github.com/drs251/c0c348e88a1cc95e3e70205bff0ed91b
def holomap_to_video(
    hmap, out=None, fps=10, dpi=200, size=None, progress_bar=util.tqdm_auto
):
    if out is None:
        out = io.BytesIO()
    renderer = hv.renderer("matplotlib")
    if dpi:
        old_dpi = renderer.dpi
        renderer.dpi = dpi
    if size:
        old_size = renderer.size
        renderer.size = size
    writer = imageio.get_writer(out, fps=fps, format="mp4")
    if isinstance(hmap, hv.Layout) or isinstance(hmap, hv.NdLayout):
        kdim = hmap[hmap.keys()[0]].kdims[0].name  # TODO: make more elegant/robust
        keys = hmap[hmap.keys()[0]].keys()
        key_items = [(k, hmap.select(**{kdim: k})) for k in keys]
    else:
        key_items = hmap.items()
    if progress_bar:
        key_items = progress_bar(key_items)
    for key, item in key_items:
        canvas = renderer.get_plot(item).state.canvas
        canvas.draw()
        ary = np.array(canvas.get_renderer()._renderer)
        writer.append_data(ary)
    writer.close()
    if dpi:
        renderer.dpi = old_dpi
    if size:
        renderer.size = old_size
    return out

In [ ]:
nd2 = nd2reader.ND2Reader("/n/scratch2/jqs1/fidelity/all/180928_txnerr_bigsnake.nd2")

In [ ]:
nd2.sizes

In [ ]:
%%time
movie = hv.HoloMap(
    {t: hv.Image(nd2.get_frame_2D(v=990, t=t, c=0)) for t in tnrange(nd2.sizes["t"])}
)
movie = movie.options("Image", framewise=True)

In [ ]:
%%output size=100
#%%opts Image {+framewise}
movie

In [ ]:
m = holomap_to_video(movie, out="/tmp/jqsmovie.mp4", size=400, dpi=100)

In [ ]:
Video("/tmp/jqsmovie.mp4", embed=True)